# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

  Using cached langchain-0.2.10-py3-none-any.whl.metadata (6.9 kB)
  Using cached langchain_core-0.2.22-py3-none-any.whl.metadata (6.0 kB)
  Using cached langsmith-0.1.93-py3-none-any.whl.metadata (13 kB)
Using cached langchain-0.2.10-py3-none-any.whl (990 kB)
Using cached langchain_core-0.2.22-py3-none-any.whl (373 kB)
Using cached langsmith-0.1.93-py3-none-any.whl (139 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.65
    Uninstalling langsmith-0.1.65:
      Successfully uninstalled langsmith-0.1.65
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.1
    Uninstalling langchain-core-0.2.1:
      Successfully uninstalled langchain-core-0.2.1
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.0
    Uninstalling langchain-0.2.0:
      Successfully uninstalled langchain-0.2.0


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [63]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


medicine_template = """你是一位医学专家。你擅长回答医学问题。
你的回答通常会包括详细的解释和建议。
当你不知道某个问题的答案时，你会建议患者去看医生。

这是一个问题：
{input}"""


chinese_template = """你是一位汉语言专家。你擅长回答汉语言问题。
你能够解释汉语言的语法和文化背景，以及汉语言中的常见误解。
你对汉语言的历史和现代用法都很了解。

这是一个问题：
{input}"""


music_template = """你是一位音乐专家。你擅长回答音乐问题。
你对中西方音乐都很了解，能够解释音乐的理论和实践。
你对唱歌技巧、乐器演奏和音乐创作都很熟悉。能够给出指导性建议。

这是一个问题：
{input}"""


computer_template = """你是一位计算机专家。你擅长回答计算机领域问题。
你对传统计算机科学、人工智能和机器学习都很了解。
你能够解释复杂的概念，并给出实用的建议。

这是一个问题：
{input}"""


biology_template = """你是一位生物专家。你擅长回答计算机领域问题。
你对传统计算机科学、人工智能和机器学习都很了解。
你能够解释复杂的概念，并给出实用的建议。

这是一个问题：
{input}"""

In [64]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "医学",
        "description": "适用于回答医学问题",
        "prompt_template": medicine_template,
    },
    {
        "name": "汉语言",
        "description": "适用于回答汉语言问题",
        "prompt_template": chinese_template,
    },
    {
        "name": "音乐",
        "description": "适用于回答音乐问题",
        "prompt_template": music_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
]

In [65]:
llm = OpenAI(api_key="sk-AlhxlLBU0BPYJD6I8870F34f377b47Dd8c1f86Ac73AfBe17",
             base_url="https://api.xiaoai.plus/v1",
             model_name="gpt-3.5-turbo-instruct")

In [66]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [67]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [68]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [69]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [70]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '医学: 适用于回答医学问题', '汉语言: 适用于回答汉语言问题', '音乐: 适用于回答音乐问题', '计算机: 适用于回答计算机问题', '生物: 适用于回答生物问题']


In [71]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
医学: 适用于回答医学问题
汉语言: 适用于回答汉语言问题
音乐: 适用于回答音乐问题
计算机: 适用于回答计算机问题
生物: 适用于回答生物问题


In [72]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [73]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
医学: 适用于回答医学问题
汉语言: 适用于回答汉语言问题
音乐: 适用于回答音乐问题
计算机: 适用于回答计算机问题
生物: 适用于回答生物问题

In [74]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [75]:
print(chain.invoke("黑体辐射是什么？?"))
print(chain.invoke("请用最诗意的方式，拒绝别人的表白。"))
print(chain.invoke("贝多芬第九交响曲想要表达什么？"))
print(chain.invoke("我的兄弟肾虚了，请问应该怎么调理？"))
print(chain.invoke("请问如何在生物实验室中制造一个生命体？"))
print(chain.invoke("如何通过AI来赚取零花钱？"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指一种理想的热辐射体，它的辐射特性只取决于它的温度，而与物体的形状、结构和成分无关。根据普朗克定律，黑体辐射的辐射强度与波长的关系是一个曲线，这条曲线的峰值波长与物体的温度有关，温度越高，峰值波长越短，辐射强度也越大。黑体辐射在研究热力学和量子力学等领域具有重要作用。'}


> Entering new MultiPromptChain chain...
None: {'input': '请用最诗意的方式，拒绝别人的表白。'}
> Finished chain.
{'input': '请用最诗意的方式，拒绝别人的表白。', 'history': '', 'text': ' 好的，我将尝试用最诗意的方式拒绝他人的表白。首先，我会向对方表达我对他们的欣赏和感激，但同时也会诚实地告诉他们我并不感兴趣。我可能会提到我对自己的未来有更重要的计划，或者我对朋友或家人的感情更加重要。我还可以说出我并不想伤害他们的感情，因此选择不接受他们的表白。总的来说，我会尽量用温柔但坚决的语言来拒绝，让对方感受到我对他们的尊重和关心。'}


> Entering new MultiPromptChain chain...
音乐: {'input': '贝多芬第九交响曲想要表达什么？'}
> Finished chain.
{'input': '贝多芬第九交响曲想要表达什么？', 'text': '\n\n贝多芬第九交响曲被认为是一部具有历史意义的作品，它不仅仅是一首音乐作品，更是一种思想的表达。在作曲家生命最后阶段，他通过这部交响曲表达了对生命的观念，对人类精神的追求和对普世和平的渴望。特别是第四乐章的《欢乐颂》，它的歌词来自德国诗人弗里德里希·席勒的诗作，表达了对人类团结和和平的向往。因此，贝多芬第九交响曲被称为“人类团结的歌曲”，它的意义远远超出了音乐本身。'}


> Entering new MultiPromptChain chain...
医学: {'input': '我的兄弟肾虚了，请问应该怎么调理？'}
> Finish

In [18]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '找到一个大于40的质数，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '找到一个大于40的质数，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要知道什么是质数。质数是指只能被1和自身整除的正整数，例如2、3、5、7等。\n\n接下来，我们需要找到一个大于40的质数。根据质数的定义，我们可以从41开始逐个往后推，直到找到一个质数。\n\n41是一个质数，但是41加一等于42，42除以3余数为0，不满足题目要求。\n\n43也是一个质数，但是43加一等于44，44除以3余数为2，同样不满足要求。\n\n47是一个大于40的质数，47加一等于48，48除以3余数为0，符合题目要求。\n\n因此，答案是47。'}


In [19]:
router_chain.verbose = True

In [20]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞'}
> Finished chain.
{'input': '什么是黑洞', 'text': '？\n\n黑洞是宇宙中非常强大的天体，它的引力非常强大，甚至连光也无法逃脱。它的存在是由于质量非常大的恒星坍塌形成的，当恒星耗尽燃料时，它就会塌缩，形成一个极为密集的物体，这就是黑洞。它的外部边界被称为事件视界，这个边界内部的一切都被吸引到黑洞内部，包括光线。因为黑洞的引力非常强大，所以它也会影响它周围的空间和物质的运动。尽管我们无法直接观测黑洞，但我们可以通过观测它周围物质的运动和辐射来推断它的存在。黑洞是宇宙中非'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains